# Exploring Neighbourhoods of Toronto

## Introduction
Toronto is the capital city of the province of Ontario in Canada. It is also the largest city in both Ontario and Canada. Found It is on the north-west side of Lake Ontario. The City of Toronto has a population of over 3 million people. Even more people live in the regions around it. All together, the Greater Toronto Area is home to over 6 million people. That makes it the biggest metropolitan area in Canada. 

## Problem Description
<p>A person has got a lucrative job offer in Toronto and has to shift with his family to Toronto.
As he is currently residing in India, he is very comfortable with the neighbourhood and  venues in India.However as he cannot reject the job offer, he wants to explore the neighbourhoods of Toronto,the venues in the neighbourhood like popular eating joints,theatres and so on.
He wants to explore those neighbourhoods which match well with his current neighbourhood.</p>

So, we need to compare different neighborhoods in terms of a service, search for potential explanation of<br> 
1) Using API, we will get the resturants,<br>
2) Getting the Indian Resturant on basis of user location, <br>
3) Using Map to check the locations of resturants. <br>


I will segment it into different neighborhoods using the geographical coordinates of the center of each neighborhood, and then using a combination of location data and machine learning, I will group the neighborhoods into clusters. 

## Target audience:
I believe this is for every person who is shifting from one place to another. As the need for a job everyone must travel and wants to be in a place where he got all the amenities. After a long hour of job, he needs that the daily needs should be nearer as much it can be. So, predicting the Battle of neighborhood is correct for description for this project. 

##  Dataset:
I would be using the Canada data to explore the neighborhood. The link for the wikipedia is 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'






In [12]:
#Installing the packages
#get_ipython().system(u' pip install --upgrade pip')
#get_ipython().system(u' pip install beautifulsoup4')
#!pip install lxml
#!pip install html5lib
#!pip install requests

#Importing packages
from bs4 import BeautifulSoup
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Reading HTML page and getting rows and columns

In [13]:
#Reading HTMl page 
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response=requests.get(url)
html = response.text
page = BeautifulSoup(html,'html.parser')
#print(page)


#Getting table from the page that is saved in page1
wiki_table = page.body.table.tbody
#print(wiki_table)

# table = page.find_all('table')[0] # Grab the first table

# definition for getting cell
def get_cell(element):
    cells = element.find_all('td')
    row = []
    
    for cell in cells:
        if cell.a:            
            if (cell.a.text):
                row.append(cell.a.text)
                continue
        row.append(cell.string.strip())
        
    return row
#Definition for getting rows.
def get_row():    
    data = []  
    
    for tr in wiki_table.find_all('tr'):
        row = get_cell(tr)
        if len(row) != 3:
            continue
        data.append(row)        
    
    return data

#### Creating a Dataframe for 3 columns PostalCode, Borough, and Neighborhood

In [5]:
rows = get_row()
columns_get = ['Postcode', 'Borough', 'Neighbourhood']
df = pd.DataFrame(rows, columns=columns_get)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Shape of the dataframe before dropping

In [6]:
df.shape

(180, 3)

#### Cleaning the data

In [7]:
df=df[df.Borough != 'Not assigned']
df = df.sort_values(by=['Postcode','Borough'])

df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)

df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Dropping the duplicates

In [8]:
df_post = df['Postcode']
df_post.drop_duplicates(inplace=True)
#df_post
df_new = pd.DataFrame(df_post)
df_new['Borough'] = '';
df_new['Neighbourhood'] = '';

In [9]:
df_new.reset_index(inplace=True)
df_new.drop('index', axis=1, inplace=True)
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)

#### Using groupby function to group the same Postcode and Borough in neighbourhood.

In [10]:
df_postcode = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_postcode.head()
locgeo_df = pd.read_csv('https://cocl.us/Geospatial_data', index_col='Postal Code')
toronto_data = df_postcode.join(locgeo_df, on='Postcode')
toronto_data.drop('Postcode', axis=1, inplace=True)
toronto_data.shape

(103, 4)

#### Getting new Dataframe shape after cleaning

In [11]:
df_postcode.shape


(103, 3)